In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

MODEL_DIR = "../models"
model_path = os.path.join(MODEL_DIR, "emotion_model.h5")
model = load_model(model_path)

label_map = {0:'angry', 1:'disgust', 2:'fear', 3:'happy', 4:'neutral', 5:'sad', 6:'surprise'}

In [17]:
from mtcnn import MTCNN

detector = MTCNN()

def predict_emotion_from_real_photo(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"⚠️ Could not load image at {image_path}")

    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb_img)

    if len(faces) == 0:
        print("⚠️ No face detected in image.")
        return

    for face in faces:
        x, y, w, h = face['box']
        roi = rgb_img[y:y+h, x:x+w]
        roi = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)
        roi = cv2.resize(roi, (48,48))
        roi = roi.astype("float32") / 255.0
        roi = np.expand_dims(roi, axis=(0,-1))

        preds = model.predict(roi)
        label = label_map[np.argmax(preds)]
        print(f"✅ Predicted emotion: {label}")

        # Draw bounding box + label
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
        cv2.putText(img, label, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,0,0), 2)

    cv2.imshow("Real Photo Prediction", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
print(os.path.exists(r"D:\Visual Studio practice\emotion_detection\Images"))
predict_emotion_from_image(r"D:\Visual Studio practice\emotion_detection\Images\angry_1.jpg")

True
1/1 [==============================] - 0s 57ms/step
✅ Predicted emotion: angry


: 

In [59]:
cap = cv2.VideoCapture(0)  # 0 = default webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Use Haar cascade to detect faces
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x,y,w,h) in faces:
        roi = gray[y:y+h, x:x+w]
        roi = cv2.resize(roi, (48,48))
        roi = roi.astype("float32") / 255.0
        roi = np.expand_dims(roi, axis=(0,-1))

        # Predict emotion
        preds = model.predict(roi)
        label = label_map[np.argmax(preds)]

        # Draw bounding box + label
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
        cv2.putText(frame, label, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,0,0), 2)

    cv2.imshow("Emotion Detection", frame)

    # Press 'q' to quit
    key = cv2.waitKey(1) & 0xFF
    if key in [ord('q'), ord('e'), ord('z')]:
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 40ms/step
